In [2]:
iris=pd.read_csv('iris.csv')
iris.head()

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa


In [3]:

import numpy as np
import random as rn 
import pandas as pd
import seaborn as sns


In [4]:
class DataProcessing:
    @staticmethod
    def shuffle(x):
        for i in range(len(x)-1,0,-1):
            j=rn.randint(0,i-1)
            x.iloc[i], x.iloc[j] = x.iloc[j], x.iloc[i]
    @staticmethod
    def normalization(x):
        #x' = (x-min)/(max-min)
        values = x.select_dtypes(exclude=["object"]) #kopia zbioru x bez nazw kwiatów
        columnNames=values.columns.tolist() #nazwy kolumn z atrybutami
        for column in columnNames:
            data = x.loc[:,column] #wszystkie rekordy z jednej z kolumn
            max1=max(data)
            min1=min(data)
            for row in x.index: #range(0, len(x),1):
                xprim=(x.at[row,column]-min1)/(max1-min1)
                x.at[row,column]=xprim


    @staticmethod
    def split(x,k): #k=0.7 -> 70% do zbioru treningowego, 30% testowy
        split_index=int(k*len(x))
        df1 = x[:split_index] #divide dataframe by rows
        df2 = x[split_index:]
        
        return df1, df2

In [5]:
DataProcessing.shuffle(iris)
# DataProcessing.normalization(iris)
trainingSet, validatingSet = DataProcessing.split(iris,0.7)
trainingSetShort = trainingSet[:5]
validatingSetShort = validatingSet[:1]
display(trainingSet.head(), validatingSet.head())

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.9,3.0,5.1,1.8,Virginica
1,5.7,4.4,1.5,0.4,Setosa
2,6.2,2.8,4.8,1.8,Virginica
3,4.5,2.3,1.3,0.3,Setosa
4,5.8,2.7,5.1,1.9,Virginica


,sepal.length,sepal.width,petal.length,petal.width,variety
105,4.9,2.5,4.5,1.7,Virginica


In [10]:
class NaiveBayes:
    @staticmethod
    def mean(column):
        return sum(column)/len(column)
    @staticmethod
    def stDeviation(column : np.array, _mean): #średnie odchylenie
        #np.sqrt(sum([(item - _mean)**2 for item in column])/len(column))
        # return np.sqrt(sum((column - _mean)**2/len(column)))
        tmp = 0
        for i in column:
            tmp += (i-_mean)**2
        return np.sqrt(tmp/len(column))
    
    @staticmethod
    def gaussProbability(x , mean, stDeviation): # def normalDist(x: np.array, mu, sigma) x in reality is np.array with 1 element 
        #https://en.wikipedia.org/wiki/Normal_distribution
        exponent=(-1/2)*((x-mean)/stDeviation)**2
        denominator = (stDeviation*np.sqrt(2*np.pi))
        base = ((1/denominator)*np.e)
        return base**exponent
    
    @staticmethod
    def classify(trainingSet, sample):
        #classes=trainingSet['variety'].unique().tolist() 
        classes=trainingSet["variety"].value_counts().index #bierzemy wszystkie klasy
        columnNames=trainingSet.select_dtypes(exclude=["object"]).columns.tolist() #kolumny bez 'variety'
        classesProducts={}
        for className in classes: #dla każdej klasy
            probabilityProduct=1    #liczymy iloczyn prawdopodobieństwa
            DataForCurrentClass=trainingSet[trainingSet['variety']==className] 
            for columnName in columnNames: #na podstawie każdej kolumny
                columnData=DataForCurrentClass.loc[:,columnName] 
                mean=NaiveBayes.mean(columnData) #liczymy składowe
                stDeviation=NaiveBayes.stDeviation(columnData, mean)
                gaussProbability=NaiveBayes.gaussProbability(sample[columnName], mean, stDeviation) #liczymy prawdopodobieństwa
                # gaussProbability=NaiveBayes.gaussProbability(columnData, mean, stDeviation) #liczymy prawdopodobieństwa
                probabilityProduct*=gaussProbability #mnożymy iloczyn
            classesProducts[className]=probabilityProduct.tolist()
        NaiveBayesClassificator=max(classesProducts, key=classesProducts.get)
        #NaiveBayesClassificatorValue=classesProducts[NaiveBayesClassificator]
        return NaiveBayesClassificator#, NaiveBayesClassificatorValue
    @staticmethod
    def calculate_positives(checked_vectors, validatingSet):
        positives = 0
        for tested_index in validatingSet.index:
                closest_variety = checked_vectors.at[tested_index, "variety"]
                true_variety = validatingSet.at[tested_index, "variety"]
                if(closest_variety == true_variety):
                    positives += 1
        return positives
    @staticmethod
    def calculate_accuracy(positives, validatingSet):
        acc=positives/len(validatingSet)
        return acc

        



  

In [7]:
# #TODO:
# 0. dla wszystkich klas wyznaczyc Naiwny klasyfikator bayesa licząc max z iloczynów prawdopodobieństw  :
# 1. dla każdej klasy policzyć iloczyn prawdopodobieństw:
# 1. dla każdej kolumny policzyć prawodpodbieństwo gausa 
# 3. dla każdego prawodpodbieństwa gaussa policzyc średnią/mediane(mu) i rozkład standardowy(sigma)
# 4. dla każdej kolumny policzyć średnią i rozkład standardowy
 


In [8]:
print(NaiveBayes.classify(trainingSet=trainingSetShort, sample=validatingSetShort))

#zapytać o przypadek kiedy np.
# sepal.length	sepal.width	petal.length	petal.width	variety
# 0	6.1	2.9	4.7	1.4	Versicolor
# 1	5.0	3.2	1.2	0.2	Setosa
# 2	6.4	3.1	5.5	1.8	Virginica
# 3	4.4	3.0	1.3	0.2	Setosa
# 4	6.1	2.8	4.0	1.3	Versicolor
# sepal.length	sepal.width	petal.length	petal.width	variety
# 105	6.4	3.2	4.5	1.5	Versicolor

# ('Versicolor', [0.0])
# /tmp/ipykernel_18995/3763426402.py:19: RuntimeWarning: divide by zero encountered in double_scalars
#   base = ((1/denominator)*np.e)

#oraz

# sepal.length	sepal.width	petal.length	petal.width	variety
# 0	0.083333	0.500000	0.067797	0.041667	Setosa
# 1	0.416667	0.833333	0.033898	0.041667	Setosa
# 2	0.444444	0.416667	0.542373	0.583333	Versicolor
# 3	0.666667	0.416667	0.677966	0.666667	Versicolor
# 4	0.388889	0.416667	0.542373	0.458333	Versicolor
# sepal.length	sepal.width	petal.length	petal.width	variety
# 105	0.611111	0.416667	0.762712	0.708333	Virginica

#wtedy output:
# ('Versicolor', [2.389652267474466e-15])
# /tmp/ipykernel_18995/3580309777.py:19: RuntimeWarning: divide by zero encountered in double_scalars
#   base = ((1/denominator)*np.e)



('Virginica', [6.603907183615538e-31])


In [9]:
#notes:
# a_test = [1,2,3]
# [item-2 for item in a_test]


#         classes=trainingSet['variety'].unique().tolist() #bierzemy wszystkie klasy
#         for className in classes: #dla każdej klasy
#             columns=validatingSet.select_dtypes(exclude=["object"]).columns().tolist() 
#             data=validatingSet[validatingSet['variety']==className]
#             for column in columns:
#                 dataFromOneColumnForOneClass=data.loc[:,column]
# myset=trainingSet.select_dtypes(exclude=["object"]).columns
# print(myset)

# classes=trainingSet["variety"].value_counts().index
# print(classes)
# columnNames=trainingSet.select_dtypes(exclude=["object"]).columns.tolist()

# print(columnNames)


In [ ]:
#lab1
#wywołanie wykresów 
#przetwarzanie danych (normalizacja) (żeby wszystkie dane były w przedziale)
#wzór x'=(x-min(x))/(max-min) można też użyć innych funkcji

#lab2
#KNN:
#1.policzyć odległości (manhatańska, minkowsky, euklidesowa?) (można losowe) 
#2.sortowanie (zmienić sposób sortowania)
#3. wzięcie k najbliższych (zmieniać k, dodać wagi np d<1- waga 1)
#4. wzrócenie wyniku

#lab3 zbiory miękkie (spodziewamy się)
#tabela klas i atrybutów
#atrybuty na 0 i 1


#lab4. NaiveBayes
#można zmienic funkcje liczenia prawdopodobieństwa
#dodać wagi

#błedy dzielić zbiory na dobre przedziały (np 0.6)


In [15]:
testingSet=validatingSet.select_dtypes(exclude=["object"])
display(testingSet.head())
display(validatingSet.head())
# varieties=[]
# for vector in testingSet:
#   varieties.append(NaiveBayes.classify(trainingSet=trainingSet, sample=validatingSet))
# testingSet['variety']=varieties
# positives=NaiveBayes.calculate_positives(testingSet, validatingSet)
# NaiveBayes.calculate_accuracy(positives, validatingSet)

,sepal.length,sepal.width,petal.length,petal.width
105,4.9,2.5,4.5,1.7


,sepal.length,sepal.width,petal.length,petal.width,variety
105,4.9,2.5,4.5,1.7,Virginica
